# Create Nightly Epoch Lightcurves

Create nightly-epoch lightcurves for "good" candidates, to use as a starting sample for transient science with the DECam deep drilling field data.

"Good" means at least 10 objects (detections in any filter) and a mean real-bogus score > 0.4 for all objects.

Create output files in the same format as made by `candidate_nightly_epochs.ipynb`:
 * candidate_lightcurves.dat
 * candidate_lightcurve_parameters.dat
 

# 0. Set up

Import packages and connect to database.

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
import pandas

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from copy import deepcopy
import time

User decat_ro, in order to access `versiontags`.

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
db.autocommit = True
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

If you want to print table schema.

In [ ]:
# tables = ['versiontags','exposures','subtractions','images',\
#           'objects','objectrbs','objectdatas','objectdata_versiontag','candidates']
# for table in tables:
#     query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
#     cursor.execute( query, ( table, ))
#     print( f"\nTABLE: {table}\n===========================" )
#     for row in cursor:
#         print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

Things in the database are tagged with versions.
This is because we might redo something, or we might try different subtraction algorithms.
This might change in the future, but, at the moment everything that's in the database should be tagged with the "latest" tag, which means it was the last thing saved to the database.

In [ ]:
tag = "latest"
db.rollback()
q = ( "SELECT id, tag FROM versiontags WHERE tag=%(tag)s" )
cursor.execute( q, { "tag": tag } )
row = cursor.fetchone()
tagid = row['id']
print(tagid)

Get all the objects in ELAIS and COSMOS fields.

This takes about a minute.

In [ ]:
%%time
q = ("SELECT od.ra, od.dec, od.mag, od.magerr, rbs.rb, "
     "i.filter, i.meanmjd, o.candidate_id, e.proposalid "
     "FROM objectdatas AS od "
     "INNER JOIN objectdata_versiontag AS odvt "
     "ON od.id=odvt.objectdata_id AND odvt.versiontag_id=%(tagid)s "
     "INNER JOIN objects AS o ON od.object_id=o.id "
     "INNER JOIN images AS i ON o.image_id=i.id "
     "INNER JOIN objectrbs as rbs ON od.id=rbs.objectdata_id AND rbs.rbtype_id=2 "
     "INNER JOIN exposures AS e ON i.exposure_id=e.id "
     "WHERE ((od.ra > 147.0 AND od.ra < 153.0 AND od.dec > -0.25 AND od.dec < 5) "
     "OR (od.ra > 5.0 AND od.ra < 12.0 AND od.dec > -46 AND od.dec < -41)) ")
cursor.execute(q, {'tagid': tagid})
df = pandas.DataFrame(cursor.fetchall())

Print total number of objects.

In [ ]:
print(len(df))

Put the objects into numpy arrays.

In [ ]:
obj_ra     = np.asarray(df[0], dtype='float')
obj_dec    = np.asarray(df[1], dtype='float')
obj_mag    = np.asarray(df[2], dtype='float')
obj_mage   = np.asarray(df[3], dtype='float')
obj_rb     = np.asarray(df[4], dtype='float')
obj_filt   = np.asarray(df[5], dtype='str')
obj_mjd    = np.asarray(df[6], dtype='float')
obj_candid = np.asarray(df[7], dtype='str')
obj_propid = np.asarray(df[8], dtype='str')
del df

The unique candidates for these objects.

In [ ]:
values, indices, counts = np.unique(obj_candid, return_index=True,  return_counts=True)

In [ ]:
print('len(values), len(indices), len(counts)', len(values), len(indices), len(counts))
print(' ')
print('On average, %4.2f objects per candidate' % (len(obj_candid)/len(values)))

<br> 

Delete all objects of candidates with just one object/candidate.

Making the object arrays smaller speeds up the processing later on.

In [ ]:
tx = np.where(counts == 1)[0]
dx = indices[tx]

In [ ]:
obj_ra     = np.delete(obj_ra, dx)
obj_dec    = np.delete(obj_dec, dx)
obj_mag    = np.delete(obj_mag, dx)
obj_mage   = np.delete(obj_mage, dx)
obj_rb     = np.delete(obj_rb, dx)
obj_filt   = np.delete(obj_filt, dx)
obj_mjd    = np.delete(obj_mjd, dx)
obj_candid = np.delete(obj_candid, dx)
obj_propid = np.delete(obj_propid, dx)
del tx, dx

In [ ]:
print(len(obj_ra))

Identify unique candidates with >= 10 objects, and calculate their mean real-bogus.

In [ ]:
tx = np.where(counts >= 10)[0]
tmp_candid = obj_candid[tx]
print(len(tx), 'candidates have >=10 objects')
del tx

In [ ]:
tmp_nobjs = np.zeros(len(tmp_candid), dtype='int')
tmp_meanrb = np.zeros(len(tmp_candid), dtype='float')
obj_keep = np.zeros(len(obj_ra), dtype='int')

This is going to take about 10 minutes.

In [ ]:
%%time
t0 = time.time()
for c, candid in enumerate(tmp_candid):
    if (c == 100) | (c == 1000) | (c == 10000):
        t1 = time.time()
        print(c, t1 - t0, 'sec')
    cx = np.where(obj_candid == candid)[0]
    tmp_nobjs[c] = len(cx)
    tmp_meanrb[c] = np.nanmean(obj_rb[cx])
    if (tmp_nobjs[c] >= 10) & (tmp_meanrb[c] >= 0.4):
        obj_keep[cx] = 1
    del cx